In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from apexpy import Apex
import lompe
from lompe.model.cmodel import Cmodel
from lompe.data_tools import dataloader
from viresclient import set_token
from scipy.signal import butter, filtfilt, freqz
from viresclient import SwarmRequest

In [2]:
set_token(
        "https://vires.services/ows",
        set_default=True,
        token="kmxv5mTTyYwzw4kQ9lsCkGfQHtjjJRVZ",
    )  # key

Token saved for https://vires.services/ows


In [3]:
from datetime import datetime
def requester(sc_collection, measurement, residual, sampling_step=None, **kwargs):
    try:
        request = SwarmRequest()
        request.set_collection(sc_collection)
        if residual == True:
            request.set_products(
                measurements=measurement,
                models=["CHAOS"],
                residuals=True,
                sampling_step=sampling_step,
            )
        else:
            request.set_products(
                measurements=measurement,
                models=["CHAOS"],
                sampling_step=sampling_step,
            )
        data = request.get_between(datetime(2022,12,19,14,2,50), datetime(2022,12,19,14,10), **kwargs) #sets to get data between the first and last value in the time tuple
        df = data.as_dataframe()
    except:
        df = []
    return df

In [4]:

def find_closest_indices(times1, times2):
    # Convert to numpy arrays
    times1 = np.array(times1)
    times2 = np.array(times2)
    
    # Compute the differences between each time in times1 and all times in times2
    # Resulting in a 2D array where each row contains the absolute differences for one time in times1
    differences = np.abs(times1[:, None] - times2)
    
    # Find the index of the minimum difference for each time in times1
    closest_indices = np.argmin(differences, axis=1)
    
    return closest_indices

In [5]:
#Swarm A's magnetics are not great but swarm B's are okay, we use this to our advantage to increase our resolution in E

measurements_E = [
        "VsatN",
        "VsatE",
        "VsatC",
        "Vixv",
        "Viy",
        "Viz",
        "Quality_flags",
        'Bx',
        'By',
        'Bz'
    ]
ds = requester( 
    "SW_EXPT_EFIB_TCT02", #Mag B, high resolution, 50Hz B (Magnetic field)
    measurements_E, #Magnetic field in NEC coordinates
    True, 
    asynchronous=False,
    show_progress=False)
dsB = requester( 
    'SW_OPER_MAGB_HR_1B', #Mag B, high resolution, 50Hz B (Magnetic field)
    ["q_NEC_CRF"], #Magnetic field in NEC coordinates
    False, 
    asynchronous=False,
    show_progress=False)


In [6]:
from scipy.spatial.transform import Rotation as R
import numpy as np

def quaternion_inverse_scipy(q):
    # Ensure q is a numpy array
    q = np.asarray(q)
    
    # Create a Rotation object from the quaternion
    rotation = R.from_quat(q)  # Note: scipy uses [x, y, z, w] format
    
    # Compute the inverse rotation
    inverse_rotation = rotation.inv()
    
    
    return inverse_rotation

In [7]:
indicies=find_closest_indices(ds.index, dsB.index)
quatnecrf=dsB["q_NEC_CRF"].to_numpy()[indicies]
quaternions = []
vsat=np.array([ds["Vixv"] , ds["Viy"], ds["Viz"]]).T
vnec=[]
for i in range(len(quatnecrf)):
    inverse_quat = quaternion_inverse_scipy(dsB["q_NEC_CRF"].to_numpy()[indicies][i])
    print(inverse_quat)
    rot_NEC_V= inverse_quat.apply(vsat[i])
    print(rot_NEC_V)
    vnec.append(rot_NEC_V)

coords=np.array([ ds['Longitude'].to_numpy(),ds['Latitude'].to_numpy()])

#Eswarm = lompe.Data(vused, coordinates = coords, datatype = 'convection', error = 50, iweight=1.0 ) 

[ 2659.15631472 -1185.56497575   410.94268209]
[2119.85496311 -726.85158853  318.99795941]
[2093.5401642  -464.27987306  259.24815435]
[1721.53759074 -709.8795615   162.90704399]
[1758.36697455 -322.75166469  166.12701218]
[1901.05051883 -601.18358866  165.82624129]
[2104.8715588  -285.27480475  393.85159121]
[2225.3136095    73.49104493  358.09630892]
[2339.44616019 -191.78517674  221.40002181]
[2393.12063529 -473.32553586  284.3195023 ]
[2177.91190088 -552.07117368  161.40819734]
[2349.02286077 -722.07142155  182.62035962]
[2333.47011923 -753.93981404   90.93701005]
[2190.63450928 -668.87949916  -41.80548001]
[2099.02814409 -727.61707405  -22.56082493]
[1973.90971549 -691.79647143  -58.24235034]
[1996.78215358 -680.04517136   24.03965737]
[1975.99024101 -753.11890772   63.90346359]
[2127.59462445 -918.46563239  146.91380528]
[ 2103.72338075 -1022.88170148   269.09385721]
[2243.55766646 -908.91071719  353.60808537]
[1938.82012213 -911.02409792  241.5021207 ]
[2011.38637528 -956.901912

In [8]:
bsat=np.array([ds["Bx"] , ds["By"], ds["Bz"]]).T
bnec=[]
for i in range(len(quatnecrf)):
    inverse_quat = quaternion_inverse_scipy(dsB["q_NEC_CRF"].to_numpy()[indicies][i])
    rot_NEC_B= inverse_quat.apply(bsat[i])
    bnec.append(rot_NEC_B)

print(bnec)

[array([ 5413.93376843,  1667.2560259 , 45827.8313986 ]), array([ 5424.6500076 ,  1671.97800786, 45825.79397374]), array([ 5435.43130011,  1675.02683712, 45823.80357021]), array([ 5446.27842667,  1676.71521322, 45821.85976622]), array([ 5456.60498803,  1679.63701589, 45819.90519802]), array([ 5466.52740802,  1683.51467477, 45817.94359521]), array([ 5477.37120914,  1686.38176994, 45815.8952998 ]), array([ 5488.93047863,  1688.46292615, 45813.78871766]), array([ 5500.65843389,  1689.56769667, 45811.66797985]), array([ 5512.51766186,  1689.91323013, 45809.53669529]), array([ 5523.52605169,  1691.42861546, 45807.48352268]), array([ 5533.87444453,  1693.8512063 , 45805.4802687 ]), array([ 5544.39200367,  1696.29680095, 45803.42487359]), array([ 5555.04282074,  1698.75931752, 45801.32937526]), array([ 5565.68327251,  1701.44930234, 45799.21388545]), array([ 5576.31680098,  1704.31551893, 45797.08244747]), array([ 5586.95121144,  1707.50782188, 45794.94235358]), array([ 5597.58566766,  1710.9

In [9]:
dsB = requester( 
    'SW_OPER_MAGB_HR_1B', #Mag B, high resolution, 50Hz B (Magnetic field)
    ["B_NEC"], #Magnetic field in NEC coordinates
    False, 
    asynchronous=False,
    show_progress=False)
print(dsB)

                                   Radius Spacecraft   Latitude  \
Timestamp                                                         
2022-12-19 14:02:50.015320301  6871771.99          B  69.827559   
2022-12-19 14:02:50.035320282  6871772.11          B  69.826297   
2022-12-19 14:02:50.055320263  6871772.23          B  69.825035   
2022-12-19 14:02:50.075320244  6871772.35          B  69.823774   
2022-12-19 14:02:50.095320225  6871772.48          B  69.822512   
...                                   ...        ...        ...   
2022-12-19 14:09:59.906023502  6875668.95          B  42.619618   
2022-12-19 14:09:59.926023483  6875669.18          B  42.618351   
2022-12-19 14:09:59.946023464  6875669.40          B  42.617084   
2022-12-19 14:09:59.966023445  6875669.63          B  42.615816   
2022-12-19 14:09:59.985953093  6875669.85          B  42.614553   

                                                                     B_NEC_CHAOS  \
Timestamp                                   